In [1]:
# Импорт библиотек
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re
import time
import numpy as np

In [2]:
test_df = pd.read_csv('test.csv')
test_df.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [4]:
test_df.brand.value_counts()

BMW           4473
VOLKSWAGEN    4404
NISSAN        4393
MERCEDES      4180
TOYOTA        3913
AUDI          3421
MITSUBISHI    2843
SKODA         2741
VOLVO         1463
HONDA         1150
INFINITI       871
LEXUS          834
Name: brand, dtype: int64

In [5]:
brand_list = [brand for brand in test_df.brand.unique()]
brand_list = [x.lower() for x in brand_list]

In [6]:
brand_list

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi']

In [7]:
def car_urls(brand):
    all_url = f'https://auto.ru/ekaterinburg/cars/{brand}/used/'   
    response = requests.get(all_url)   
    soap = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')   
    find_count = soap.find('span', class_='ButtonWithLoader__content').text.replace(u'\xa0', '')
    urls_count = int(re.findall(r'\d+', find_count)[0])
    count_per_page = len(soap.find_all('a', class_='Link ListingItemTitle__link'))
    pages_num = urls_count // count_per_page
    list_urls = []
    for page_num in range(1, pages_num):
        if page_num % 20 == 0:
            print(f"Извлечено {page_num} из {pages_num}")
        page_url = f'https://auto.ru/ekaterinburg/cars/{brand}/used/?page={page_num}'   
        page_response = requests.get(page_url)
        time.sleep(0.1)
        page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
        for a in page_soap.find_all('a', class_='Link ListingItemTitle__link'):
            list_urls.append(a.get('href'))
    return list_urls

In [8]:
#urls_count = int(re.findall(r'\d+', find_count)[0])
#count_per_page = len(soap.find_all('a', class_='Link ListingItemTitle__link'))
#pages_num_skoda = urls_count // count_per_page
#skoda_urls = []
#for page_num in range(1, pages_num_skoda):
#    if page_num % 20 == 0:
#        print(f"Извлечено {page_num} из {pages_num_skoda}")
#    page_url = f'https://auto.ru/cars/skoda/all/?page={page_num}'   
#    page_response = requests.get(page_url)
#    time.sleep(0.1)
#    page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
#    for a in page_soap.find_all('a', class_='Link ListingItemTitle__link'):
#       skoda_urls.append(a.get('href'))
#len(skoda_urls)

In [9]:
#skoda_urls = []
#for page_num in range(1, pages_num_skoda):
#    if page_num % 20 == 0:
#        print(f"Extracting page {page_num} from {pages_num_skoda}")
#    page_url = f'https://auto.ru/cars/skoda/all/?page={page_num}'   
#    page_response = requests.get(page_url)
#    time.sleep(0.1)
#    page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
#    for a in page_soap.find_all('a', class_='Link ListingItemTitle__link'):
#       skoda_urls.append(a.get('href'))
#len(skoda_urls)

In [10]:
urls_list = []
for brand in brand_list:
    print(f"Extracting data for the brand {brand}:")
    urls_list.extend(car_urls(brand))

Extracting data for the brand skoda:
Extracting data for the brand audi:
Extracting data for the brand honda:
Extracting data for the brand volvo:
Extracting data for the brand bmw:
Extracting data for the brand nissan:
Извлечено 20 из 23
Extracting data for the brand infiniti:
Extracting data for the brand mercedes:
Extracting data for the brand toyota:
Извлечено 20 из 32
Extracting data for the brand lexus:
Extracting data for the brand volkswagen:
Извлечено 20 из 28
Extracting data for the brand mitsubishi:


In [11]:
new_train_df = pd.DataFrame({'car_url': urls_list})
new_train_df.to_csv('new_train_ekat.csv', index=False)

In [12]:
# new_train_df = pd.read_csv('new_train_ekat.csv')

In [13]:
new_train_df

,car_url
0,https://auto.ru/cars/used/sale/skoda/octavia/1...
1,https://auto.ru/cars/used/sale/skoda/rapid/110...
2,https://auto.ru/cars/used/sale/skoda/fabia/110...
3,https://auto.ru/cars/used/sale/skoda/roomster/...
4,https://auto.ru/cars/used/sale/skoda/octavia/1...
...,...
5521,https://auto.ru/cars/used/sale/mitsubishi/paje...
5522,https://auto.ru/cars/used/sale/mitsubishi/lanc...
5523,https://auto.ru/cars/used/sale/mitsubishi/deli...
5524,https://auto.ru/cars/used/sale/mitsubishi/asx/...


In [14]:
new_train_df['brand'] =0
new_train_df

,car_url,brand
0,https://auto.ru/cars/used/sale/skoda/octavia/1...,0
1,https://auto.ru/cars/used/sale/skoda/rapid/110...,0
2,https://auto.ru/cars/used/sale/skoda/fabia/110...,0
3,https://auto.ru/cars/used/sale/skoda/roomster/...,0
4,https://auto.ru/cars/used/sale/skoda/octavia/1...,0
...,...,...
5521,https://auto.ru/cars/used/sale/mitsubishi/paje...,0
5522,https://auto.ru/cars/used/sale/mitsubishi/lanc...,0
5523,https://auto.ru/cars/used/sale/mitsubishi/deli...,0
5524,https://auto.ru/cars/used/sale/mitsubishi/asx/...,0


In [15]:
for i in range(0, len(new_train_df)):
    if new_train_df.car_url[i].find('skoda') > 0:
        new_train_df.brand[i] = 'skoda'
    if new_train_df.car_url[i].find('audi') > 0:
        new_train_df.brand[i] = 'audi'
    if new_train_df.car_url[i].find('honda') > 0:
        new_train_df.brand[i] = 'honda'
    if new_train_df.car_url[i].find('volvo') > 0:
        new_train_df.brand[i] = 'volvo'
    if new_train_df.car_url[i].find('bmw') > 0:
        new_train_df.brand[i] = 'bmw'
    if new_train_df.car_url[i].find('nissan') > 0:
        new_train_df.brand[i] = 'nissan'
    if new_train_df.car_url[i].find('infiniti') > 0:
        new_train_df.brand[i] = 'infiniti'
    if new_train_df.car_url[i].find('mercedes') > 0:
        new_train_df.brand[i] = 'mercedes'
    if new_train_df.car_url[i].find('toyota') > 0:
        new_train_df.brand[i] = 'toyota'
    if new_train_df.car_url[i].find('lexus') > 0:
        new_train_df.brand[i] = 'lexus'
    if new_train_df.car_url[i].find('volkswagen') > 0:
        new_train_df.brand[i] = 'volkswagen'
    if new_train_df.car_url[i].find('mitsubishi') > 0:
        new_train_df.brand[i] = 'mitsubishi'

<ipython-input-15-9a9ad4b68e50>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train_df.brand[i] = 'skoda'
C:\Users\cloud\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
extracted_columns = test_df.columns.to_list()

In [17]:
extracted_columns.remove('car_url')
extracted_columns.remove('brand')
extracted_columns.append('price')
extracted_columns

['bodyType',
 'color',
 'complectation_dict',
 'description',
 'engineDisplacement',
 'enginePower',
 'equipment_dict',
 'fuelType',
 'image',
 'mileage',
 'modelDate',
 'model_info',
 'model_name',
 'name',
 'numberOfDoors',
 'parsing_unixtime',
 'priceCurrency',
 'productionDate',
 'sell_id',
 'super_gen',
 'vehicleConfiguration',
 'vehicleTransmission',
 'vendor',
 'Владельцы',
 'Владение',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня',
 'price']

In [18]:
for cols in extracted_columns:
    new_train_df[cols] = ''
new_train_df

,car_url,brand,bodyType,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,https://auto.ru/cars/used/sale/skoda/octavia/1...,skoda,,,,,,,,,...,,,,,,,,,,
1,https://auto.ru/cars/used/sale/skoda/rapid/110...,skoda,,,,,,,,,...,,,,,,,,,,
2,https://auto.ru/cars/used/sale/skoda/fabia/110...,skoda,,,,,,,,,...,,,,,,,,,,
3,https://auto.ru/cars/used/sale/skoda/roomster/...,skoda,,,,,,,,,...,,,,,,,,,,
4,https://auto.ru/cars/used/sale/skoda/octavia/1...,skoda,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5521,https://auto.ru/cars/used/sale/mitsubishi/paje...,mitsubishi,,,,,,,,,...,,,,,,,,,,
5522,https://auto.ru/cars/used/sale/mitsubishi/lanc...,mitsubishi,,,,,,,,,...,,,,,,,,,,
5523,https://auto.ru/cars/used/sale/mitsubishi/deli...,mitsubishi,,,,,,,,,...,,,,,,,,,,
5524,https://auto.ru/cars/used/sale/mitsubishi/asx/...,mitsubishi,,,,,,,,,...,,,,,,,,,,


In [19]:
for i in range(0, len(new_train_df.car_url)):
    try:
        response=requests.get(new_train_df.car_url[i])
        if response.status_code == 200:
            try:
                soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
                data = json.loads(soup.find('script', type="application/ld+json").string)
                json_data_equip = json.loads(soup.find('script', type="application/json", id='initial-state').string)
            except:
                pass
            try:
                new_train_df.bodyType[i]=json_data_equip['card']['vehicle_info']['configuration']['human_name']
            except:
                pass
            try:
                new_train_df.productionDate[i]=data['productionDate']
            except:
                pass
            try:
                new_train_df.color[i]=data['color']
            except:
                pass
            try:
                new_train_df.equipment_dict[i]=json_data_equip['card']['vehicle_info']['equipment']
            except:
                pass
            try:
                new_train_df.model_name[i]=data['name']
            except:
                pass
            try:
                new_train_df.description[i]=data['description']
            except:
                pass
            try:
                new_train_df['Владельцы'][i]=json_data_equip['card']['documents']['owners_number']
            except:
                pass
            try:
                new_train_df.enginePower[i]=json_data_equip['card']['owner_expenses']['transport_tax']['horse_power']
            except:
                pass
            try:
                new_train_df.engineDisplacement[i]=json_data_equip['card']['vehicle_info']['tech_param']['human_name']
            except:
                pass
            try:
                fuel = soup.find('li', class_='CardInfoRow CardInfoRow_engine').find_all('span')[-1].text.replace(u'\xa0', u'')
                new_train_df.fuelType[i]=re.findall(r'\w+[а-яА-ЯёЁ]',fuel)[-1]
            except:
                pass
            try:
                mileage = soup.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[-1].text.replace(u'\xa0', u'')
                new_train_df.mileage[i]= int(re.findall(r'\d+', mileage)[0])
            except:
                pass
            try:
                new_train_df.modelDate[i]=json_data_equip['card']['vehicle_info']['super_gen']['year_from']
            except:
                pass
            try:
                new_train_df.model_name[i]=json_data_equip['card']['vehicle_info']['model_info']['code']
            except:
                pass
            try:
                new_train_df.numberOfDoors[i]=json_data_equip['card']['vehicle_info']['configuration']['doors_count']
            except:
                pass
            try:
                new_train_df.parsing_unixtime[i]=int(time.time())
            except:
                pass
            try:
                new_train_df.priceCurrency[i]=json_data_equip['card']['price_info']['currency']
            except:
                pass
            try:
                new_train_df.super_gen[i]=json_data_equip['card']['vehicle_info']['tech_param']
            except:
                pass
            try:
                new_train_df.vehicleTransmission[i]=soup.find(
                    'li', class_='CardInfoRow CardInfoRow_transmission').find_all('span')[-1].text
            except:
                pass
            try:
                new_train_df.vendor[i]=json_data_equip['card']['vehicle_info']['vendor']
            except:
                pass
            try:
                new_train_df['ПТС'][i]=soup.find(
                    'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
            except:
                pass
            try:
                new_train_df['Привод'][i]=soup.find(
                    'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
            except:
                pass
            try:
                new_train_df['Руль'][i]=soup.find(
                    'li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
            except:
                pass
            try:
                new_train_df['Состояние'][i]=soup.find(
                    'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
            except:
                pass
            try:
                new_train_df['Таможня'][i]=soup.find(
                    'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
            except:
                pass
            try:
                new_train_df.price[i]=json_data_equip['card']['price_info']['price']
            except:
                pass
            try:
                new_train_df.to_csv('new_train_ekat.csv', index=False)
                time.sleep(1.5)
            except:
                pass
    except:
        pass

In [20]:
new_train_df

,car_url,brand,bodyType,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,https://auto.ru/cars/used/sale/skoda/octavia/1...,skoda,Лифтбек,серый,,"Продам новый автомобиль, покупался и обслужива...",1.4 AT (150 л.с.),150,"{'cruise-control': True, 'asr': True, 'tinted-...",Бензин,...,автоматическая,EUROPEAN,1,,Оригинал,передний,Левый,Не требует ремонта,Растаможен,2240000
1,https://auto.ru/cars/used/sale/skoda/rapid/110...,skoda,Лифтбек,белый,,Продам практичный и комфортный автомобиль в ид...,1.4 AMT (125 л.с.),125,"{'engine-proof': True, 'driver-seat-support': ...",Бензин,...,роботизированная,EUROPEAN,1,,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1010000
2,https://auto.ru/cars/used/sale/skoda/fabia/110...,skoda,Универсал 5 дв.,чёрный,,"Официальный дилер Skoda, компания ""Уникум"" г. ...",1.4 MT (86 л.с.),86,"{'airbag-driver': True, 'isofix': True, 'elect...",Бензин,...,механическая,EUROPEAN,1,,Оригинал,передний,Левый,Не требует ремонта,Растаможен,700000
3,https://auto.ru/cars/used/sale/skoda/roomster/...,skoda,Компактвэн,белый,,Автосалон Carbike предлагает Вашему вниманию S...,1.6 MT (105 л.с.),105,"{'cruise-control': True, 'asr': True, 'tinted-...",Бензин,...,механическая,EUROPEAN,2,,Оригинал,передний,Левый,Не требует ремонта,Растаможен,860000
4,https://auto.ru/cars/used/sale/skoda/octavia/1...,skoda,Лифтбек,белый,,НАШИ АВТОМОБИЛИ:\n• Прошли полную техническую...,1.6 MT (110 л.с.),110,"{'electro-window-back': True, 'ptf': True, 'es...",Бензин,...,механическая,EUROPEAN,2,,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1820000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5521,https://auto.ru/cars/used/sale/mitsubishi/paje...,mitsubishi,Внедорожник 5 дв.,серебристый,,"Авто на ходу, но требуется ремонт.\nПлюсом к м...",2.0 AT (136 л.с.),136,"{'airbag-passenger': True, 'abs': True, 'feedb...",Бензин,...,автоматическая,JAPANESE,2,,Оригинал,задний,Правый,Не требует ремонта,Растаможен,270000
5522,https://auto.ru/cars/used/sale/mitsubishi/lanc...,mitsubishi,Универсал 5 дв.,серебристый,,"Отличный полноприводный японец, много уже сдел...",1.6 AT (113 л.с.) 4WD,113,"{'ptf': True, 'aux': True, 'usb': True, 'turnb...",Бензин,...,автоматическая,JAPANESE,3,,Дубликат,полный,Левый,Не требует ремонта,Растаможен,160000
5523,https://auto.ru/cars/used/sale/mitsubishi/deli...,mitsubishi,Минивэн,чёрный,,"На ходу , требует ремонт по кузову,двиготель к...",2.4 MT (115 л.с.) 4WD,115,"{'tinted-glass': True, 'velvet-seats': True, '...",Бензин,...,механическая,JAPANESE,3,,Оригинал,полный,Правый,Не требует ремонта,Растаможен,290000
5524,https://auto.ru/cars/used/sale/mitsubishi/asx/...,mitsubishi,Внедорожник 5 дв.,белый,,1 собственник!!!\nПродаю автомобиль в техничес...,1.6d MT (114 л.с.),114,{},Дизель,...,механическая,JAPANESE,1,,Оригинал,передний,Левый,Не требует ремонта,Растаможен,960000
